In [33]:
from typing import Dict, List, Optional
from datetime import datetime
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import StructuredTool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
# 변경 전:
# from langchain.prompts.chat import SystemMessage, HumanMessage

# 변경 후:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
class LocationSchema(BaseModel):
    nx: int
    ny: int

class WeatherReporterAgent:
    def __init__(self, weather_client, openai_api_key: str):
        self.weather_client = weather_client
        self.llm = ChatOpenAI(
            temperature=0.7,
            model='gpt-4',
            openai_api_key=openai_api_key
        )
        self.setup_agent()

    def get_all_forecasts(self, location: LocationSchema) -> Dict:
        """모든 예보 데이터를 수집하고 통합"""
        current_time = datetime.now()
        base_date = current_time.strftime("%Y%m%d")
        base_time = current_time.strftime("%H%M")

        current = self.weather_client.get_ultra_srt_ncst(
            base_date=base_date,
            base_time=self._adjust_base_time(base_time, interval=60),
            nx=location.nx,
            ny=location.ny
        )

        ultra_short = self.weather_client.get_ultra_srt_fcst(
            base_date=base_date,
            base_time=self._adjust_base_time(base_time, interval=30),
            nx=location.nx,
            ny=location.ny
        )

        short = self.weather_client.get_vilage_fcst(
            base_date=base_date,
            base_time=self._get_latest_short_forecast_time(current_time),
            nx=location.nx,
            ny=location.ny
        )

        return {
            'current': self._parse_items(current),
            'ultra_short': self._parse_items(ultra_short),
            'short': self._parse_items(short)
        }
            
    def setup_agent(self):
        tools = [
            StructuredTool.from_function(
                func=self.get_all_forecasts,
                name="get_weather_forecast",
                description="Get weather forecast data",
                args_schema=LocationSchema
            )
        ]

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a professional Korean weather forecaster. 
    Available Tools: {tools}
    Tool Names: {tool_names}

    When providing forecasts:
    1. Format time periods cohesively
    2. Highlight significant changes
    3. Include practical recommendations
    4. Use professional but friendly tone

    Respond in Korean."""),
            HumanMessage(content="{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])

        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        agent = create_react_agent(
            llm=self.llm,
            tools=tools,
            prompt=prompt
        )
        
        self.agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True
        )
            
    def get_weather_report(self, nx: int, ny: int, focus: Optional[str] = None) -> str:
        question = f"위치(nx={nx}, ny={ny})의 날씨 정보를 분석하고, "
        if focus:
            question += f"특히 {focus}에 중점을 두어 "
        question += "기상캐스터처럼 자연스럽게 전달해주세요."
        
        return self.agent_executor.invoke({"input": question})["output"]

    # Helper methods remain unchanged
    def _parse_items(self, response: Dict) -> List[Dict]:
        if 'items' in response and 'item' in response['items']:
            return response['items']['item']
        return []

    def _adjust_base_time(self, time: str, interval: int) -> str:
        hour = int(time[:2])
        minute = int(time[2:])
        total_minutes = hour * 60 + minute
        adjusted_minutes = (total_minutes // interval) * interval
        adjusted_hour = adjusted_minutes // 60
        adjusted_minute = adjusted_minutes % 60
        return f"{adjusted_hour:02d}{adjusted_minute:02d}"

    def _get_latest_short_forecast_time(self, current_time: datetime) -> str:
        forecast_hours = [2, 5, 8, 11, 14, 17, 20, 23]
        current_hour = current_time.hour
        
        for hour in reversed(forecast_hours):
            if current_hour > hour:
                return f"{hour:02d}00"
        
        return "2300"

In [ ]:
import requests
from datetime import datetime
from typing import Dict, Any, Optional

class WeatherAPIClient:
    def __init__(self, service_key: str, base_url: str = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0"):
        self.service_key = service_key
        self.base_url = base_url

    def _make_request(self, endpoint: str, params: Dict[str, Any]) -> Dict:
        """Make API request with error handling"""
        params['serviceKey'] = self.service_key
        
        url = f"{self.base_url}/{endpoint}"
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        result = response.json()
        header = result['response']['header']
        
        if header['resultCode'] != '00':
            raise Exception(f"API Error: {header['resultCode']} - {header['resultMsg']}")
            
        return result['response']['body']

    def get_ultra_srt_ncst(self, base_date: str, base_time: str, nx: int, ny: int, 
                          data_type: str = 'JSON', num_of_rows: int = 10, page_no: int = 1) -> Dict:
        """Get ultra short-term forecast status"""
        params = {
            'numOfRows': num_of_rows,
            'pageNo': page_no,
            'dataType': data_type,
            'base_date': base_date,
            'base_time': base_time,
            'nx': nx,
            'ny': ny
        }
        return self._make_request('getUltraSrtNcst', params)

    def get_ultra_srt_fcst(self, base_date: str, base_time: str, nx: int, ny: int,
                          data_type: str = 'JSON', num_of_rows: int = 10, page_no: int = 1) -> Dict:
        """Get ultra short-term forecast"""
        params = {
            'numOfRows': num_of_rows,
            'pageNo': page_no,
            'dataType': data_type,
            'base_date': base_date,
            'base_time': base_time,
            'nx': nx,
            'ny': ny
        }
        return self._make_request('getUltraSrtFcst', params)

    def get_vilage_fcst(self, base_date: str, base_time: str, nx: int, ny: int,
                       data_type: str = 'JSON', num_of_rows: int = 10, page_no: int = 1) -> Dict:
        """Get short-term forecast"""
        params = {
            'numOfRows': num_of_rows,
            'pageNo': page_no,
            'dataType': data_type,
            'base_date': base_date,
            'base_time': base_time,
            'nx': nx,
            'ny': ny
        }
        return self._make_request('getVilageFcst', params)

    def get_fcst_version(self, ftype: str, base_datetime: str,
                        data_type: str = 'JSON', num_of_rows: int = 10, page_no: int = 1) -> Dict:
        """Get forecast version information"""
        params = {
            'numOfRows': num_of_rows,
            'pageNo': page_no,
            'dataType': data_type,
            'ftype': ftype,
            'basedatetime': base_datetime
        }
        return self._make_request('getFcstVersion', params)

# Helper class for weather data interpretation
class WeatherCodes:
    SKY_CODES = {
        1: "맑음",
        3: "구름많음", 
        4: "흐림"
    }
    
    PTY_CODES_ULTRA = {
        0: "없음",
        1: "비",
        2: "비/눈",
        3: "눈",
        5: "빗방울",
        6: "빗방울눈날림",
        7: "눈날림"
    }
    
    PTY_CODES_VILLAGE = {
        0: "없음",
        1: "비",
        2: "비/눈",
        3: "눈",
        4: "소나기"
    }


'\nclient = WeatherAPIClient("your-service-key")\n\n# Get current weather status\nresult = client.get_ultra_srt_ncst(\n    base_date="20240315",\n    base_time="1400",\n    nx=55,\n    ny=127\n)\n\n# Get ultra short term forecast\nforecast = client.get_ultra_srt_fcst(\n    base_date="20240315",\n    base_time="1430",\n    nx=55,\n    ny=127\n)\n\n# Get village forecast\nvillage = client.get_vilage_fcst(\n    base_date="20240315",\n    base_time="0500",\n    nx=55,\n    ny=127\n)\n\n# Get forecast version\nversion = client.get_fcst_version(\n    ftype="ODAM",\n    base_datetime="202403150800"\n)\n'

In [35]:

import os
import json
from datetime import datetime

def get_weather_report(api_key: str, location: dict) -> dict:
    try:
        # API 클라이언트와 리포터 초기화
        weather_client = WeatherAPIClient(api_key)
        reporter = WeatherReporterAgent(weather_client, api_key)

        # 일반 날씨 리포트
        general_report = reporter.get_weather_report(
            nx=location['nx'], 
            ny=location['ny']
        )

        # 강수 관련 상세 리포트
        rain_report = reporter.get_weather_report(
            nx=location['nx'],
            ny=location['ny'],
            focus="강수 확률과 우산 필요성"
        )

        return {
            'timestamp': datetime.now().isoformat(),
            'location': location,
            'general_report': general_report,
            'rain_report': rain_report
        }

    except Exception as e:
        return {
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        }



In [36]:

api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

# 서울 강남구 좌표
location = {
    'name': '서울 강남구',
    'nx': 62,
    'ny': 126
}

result = get_weather_report(api_key, location)
print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "error": "Prompt missing required variables: {'tools', 'tool_names'}",
  "timestamp": "2024-11-15T23:38:51.113175"
}


In [52]:
from typing import Dict, List, Optional
from datetime import datetime
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import StructuredTool
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage, HumanMessage

class LocationSchema(BaseModel):
    nx: int
    ny: int

class WeatherReporterAgent:
    def __init__(self, weather_client, openai_api_key: str):
        self.weather_client = weather_client
        self.llm = ChatOpenAI(
            temperature=0.7,
            model='gpt-4',
            openai_api_key=openai_api_key
        )
        self.setup_agent()

    def get_all_forecasts(self, location: LocationSchema) -> Dict:
        """모든 예보 데이터를 수집하고 통합"""
        current_time = datetime.now()
        base_date = current_time.strftime("%Y%m%d")
        base_time = current_time.strftime("%H%M")

        current = self.weather_client.get_ultra_srt_ncst(
            base_date=base_date,
            base_time=self._adjust_base_time(base_time, interval=60),
            nx=location.nx,
            ny=location.ny
        )

        ultra_short = self.weather_client.get_ultra_srt_fcst(
            base_date=base_date,
            base_time=self._adjust_base_time(base_time, interval=30),
            nx=location.nx,
            ny=location.ny
        )

        short = self.weather_client.get_vilage_fcst(
            base_date=base_date,
            base_time=self._get_latest_short_forecast_time(current_time),
            nx=location.nx,
            ny=location.ny
        )

        return {
            'current': self._parse_items(current),
            'ultra_short': self._parse_items(ultra_short),
            'short': self._parse_items(short)
        }
        
    def setup_agent(self):
        tool = StructuredTool.from_function(
            func=self.get_all_forecasts,
            name="get_weather_forecast",
            description="Get weather forecast data",
            args_schema=LocationSchema
        )

        tools = [tool]
        
        tools_str = "\n".join([f"- {t.name}: {t.description}" for t in tools])
        tool_names = ", ".join([t.name for t in tools])

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=f"""당신은 전문 한국 기상 캐스터입니다. 
    예보를 제공할 때:
    1. 시간대를 일관되게 포맷하세요.
    2. 중요한 변화를 강조하세요.
    3. 실용적인 추천을 포함하세요.
    4. 전문적이지만 친근한 어조를 사용하세요.

    사용 가능한 도구:
    {tools_str}

    도구 이름: {tool_names}

    한국어로 응답하세요."""),
            HumanMessage(content="{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

        # 프롬프트에 필수 변수를 포함하도록 수정
        prompt = prompt.partial(tools=tools_str, tool_names=tool_names)

        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        # 에이전트 생성 시 프롬프트에 변수를 포함
        self.agent = create_react_agent(
            llm=self.llm,
            tools=tools,
            prompt=prompt
        )
        
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=tools,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True
        )
        
    def get_weather_report(self, nx: int, ny: int, focus: Optional[str] = None) -> str:
        question = f"위치(nx={nx}, ny={ny})의 날씨 정보를 분석하고, "
        if focus:
            question += f"특히 {focus}에 중점을 두어 "
        question += "기상캐스터처럼 자연스럽게 전달해주세요."
        
        return self.agent_executor.invoke({"input": question})["output"]

    def _parse_items(self, response: Dict) -> List[Dict]:
        if 'items' in response and 'item' in response['items']:
            return response['items']['item']
        return []

    def _adjust_base_time(self, time: str, interval: int) -> str:
        hour = int(time[:2])
        minute = int(time[2:])
        total_minutes = hour * 60 + minute
        adjusted_minutes = (total_minutes // interval) * interval
        adjusted_hour = adjusted_minutes // 60
        adjusted_minute = adjusted_minutes % 60
        return f"{adjusted_hour:02d}{adjusted_minute:02d}"

    def _get_latest_short_forecast_time(self, current_time: datetime) -> str:
        forecast_hours = [2, 5, 8, 11, 14, 17, 20, 23]
        current_hour = current_time.hour
        
        for hour in reversed(forecast_hours):
            if current_hour > hour:
                return f"{hour:02d}00"
        
        return "2300"

ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
import os
from typing import Dict, List, Optional
from datetime import datetime
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import StructuredTool
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage, HumanMessage

class LocationSchema(BaseModel):
    nx: int
    ny: int

class WeatherReporterAgent:
    def __init__(self, weather_client, openai_api_key: str):
        self.weather_client = weather_client
        self.llm = ChatOpenAI(
            temperature=0.7,
            model='gpt-4',
            openai_api_key=openai_api_key
        )
        self.setup_agent()

    def get_all_forecasts(self, location: LocationSchema) -> Dict:
        """모든 예보 데이터를 수집하고 통합"""
        current_time = datetime.now()
        base_date = current_time.strftime("%Y%m%d")
        base_time = current_time.strftime("%H%M")

        current = self.weather_client.get_ultra_srt_ncst(
            base_date=base_date,
            base_time=self._adjust_base_time(base_time, interval=60),
            nx=location.nx,
            ny=location.ny
        )

        ultra_short = self.weather_client.get_ultra_srt_fcst(
            base_date=base_date,
            base_time=self._adjust_base_time(base_time, interval=30),
            nx=location.nx,
            ny=location.ny
        )

        short = self.weather_client.get_vilage_fcst(
            base_date=base_date,
            base_time=self._get_latest_short_forecast_time(current_time),
            nx=location.nx,
            ny=location.ny
        )

        return {
            'current': self._parse_items(current),
            'ultra_short': self._parse_items(ultra_short),
            'short': self._parse_items(short)
        }
            
    def setup_agent(self):
        tool = StructuredTool.from_function(
            func=self.get_all_forecasts,
            name="get_weather_forecast",
            description="Fetches comprehensive weather forecast data",
            args_schema=LocationSchema
        )

        tools = [tool]
        
        tools_str = "\n".join([f"- {t.name}: {t.description}" for t in tools])
        tool_names = ", ".join([t.name for t in tools])

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=f"""당신은 전문 한국 기상 캐스터입니다. 
예보를 제공할 때:
1. 시간대를 일관되게 포맷하세요.
2. 중요한 변화를 강조하세요.
3. 실용적인 추천을 포함하세요.
4. 전문적이지만 친근한 어조를 사용하세요.

사용 가능한 도구:
{tools_str}

도구 이름: {tool_names}

한국어로 응답하세요."""),
            HumanMessage(content="{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")  # 올바르게 설정
        ])

        # 프롬프트에 필수 변수를 포함하도록 수정
        prompt = prompt.partial(tools=tools_str, tool_names=tool_names)

        self.memory = ConversationBufferMemory(
            memory_key="agent_scratchpad",  # 메모리 키를 agent_scratchpad로 설정
            return_messages=True
        )
        
        self.agent = create_react_agent(
            llm=self.llm,
            tools=tools,
            prompt=prompt
        )
        
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=tools,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True
        )
            
    def get_weather_report(self, nx: int, ny: int, focus: Optional[str] = None) -> str:
        question = f"위치(nx={nx}, ny={ny})의 날씨 정보를 분석하고, "
        if focus:
            question += f"특히 {focus}에 중점을 두어 "
        question += "기상캐스터처럼 자연스럽게 전달해주세요."
        
        return self.agent_executor.invoke({"input": question})["output"]

    def _parse_items(self, response: Dict) -> List[Dict]:
        if 'items' in response and 'item' in response['items']:
            return response['items']['item']
        return []

    def _adjust_base_time(self, time: str, interval: int) -> str:
        hour = int(time[:2])
        minute = int(time[2:])
        total_minutes = hour * 60 + minute
        adjusted_minutes = (total_minutes // interval) * interval
        adjusted_hour = adjusted_minutes // 60
        adjusted_minute = adjusted_minutes % 60
        return f"{adjusted_hour:02d}{adjusted_minute:02d}"

    def _get_latest_short_forecast_time(self, current_time: datetime) -> str:
        forecast_hours = [2, 5, 8, 11, 14, 17, 20, 23]
        current_hour = current_time.hour
        
        for hour in reversed(forecast_hours):
            if current_hour > hour:
                return f"{hour:02d}00"
        
        return "2300"

# API 키 설정
openai_api_key = os.getenv('OPENAI_API_KEY')
weather_api_key = os.getenv('KMA_API_KEY')

if not all([openai_api_key, weather_api_key]):
    raise ValueError("Required API keys not found in environment variables")

# 클라이언트 및 에이전트 초기화
weather_client = WeatherAPIClient(weather_api_key)
weather_agent = WeatherReporterAgent(weather_client, openai_api_key)

# 서울 강남구 좌표로 날씨 정보 요청
try:
    general_report = weather_agent.get_weather_report(nx=62, ny=126)
    print("\n일반 날씨 리포트:")
    print(general_report)
    
    rain_report = weather_agent.get_weather_report(
        nx=62, 
        ny=126, 
        focus="강수 확률과 우산 필요성"
    )
    print("\n강수 관련 리포트:")
    print(rain_report)
    
except Exception as e:
    import traceback
    traceback.print_exc()
    print(f"Error occurred: {str(e)}")




> Entering new AgentExecutor chain...
Error occurred: variable agent_scratchpad should be a list of base messages, got  of type <class 'str'>
